# Engineering Salary Data Analysis

This notebook performs exploratory data analysis on engineering salary data to understand the key factors affecting compensation and identify patterns in the dataset.

In [ ]:
pip install missingno

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import missingno as msno

# Set visualization styles
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load and Examine the Data

First, let's load the CSV file and take a look at the first few rows, shape, and basic information about the dataset.

In [ ]:
# Load the dataset
df = pd.read_csv('data/train.csv')

# Display the first few rows
print("First 5 rows of the dataset:")
display(df.head())

# Display the shape of the dataset
print(f"\nDataset dimensions: {df.shape[0]} rows and {df.shape[1]} columns")

## 2. Data Overview and Statistics

Let's get a comprehensive overview of the dataset including data types, summary statistics, and column descriptions.

In [ ]:
# Check the data types of each column
print("Data types of each column:")
display(df.dtypes)

# Generate basic statistics for numerical columns
print("\nSummary statistics for numerical columns:")
display(df.describe().T)

# Display information about the dataset
print("\nDataset information:")
df.info()

# Check for duplicate rows
print(f"\nNumber of duplicate rows: {df.duplicated().sum()}")

# List all column names
print("\nColumn names:")
for col in df.columns:
    print(f"- {col}")

## 3. Missing Value Analysis

Let's check for missing values in the dataset and visualize their distribution across features.

In [ ]:
# Calculate missing values per column
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

# Create a dataframe to display missing values statistics
missing_data = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage.round(2)
})

# Display columns with missing values
print("Columns with missing values:")
display(missing_data[missing_data['Missing Values'] > 0].sort_values(by='Missing Values', ascending=False))

# Visualize missing values
plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), yticklabels=False, cbar=False, cmap='viridis')
plt.title('Missing Value Heatmap')
plt.tight_layout()
plt.show()

# Create a bar chart of missing values by column
missing_data_nonzero = missing_data[missing_data['Missing Values'] > 0].sort_values(by='Missing Values')
if not missing_data_nonzero.empty:
    plt.figure(figsize=(10, 6))
    plt.barh(missing_data_nonzero.index, missing_data_nonzero['Percentage'])
    plt.title('Percentage of Missing Values by Column')
    plt.xlabel('Percentage')
    plt.tight_layout()
    plt.show()

    # Visualize missing values with missingno
    msno.matrix(df)
    plt.title('Missing Values Matrix')
    plt.show()
    
    msno.bar(df)
    plt.title('Missing Values Bar Chart')
    plt.show()
else:
    print("No missing values in the dataset.")

## 4. Numerical Features Analysis

Let's analyze the distribution of numerical features using histograms, box plots, and descriptive statistics.

In [ ]:
# Identify numerical columns
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f"Numerical columns: {numerical_columns}")

# Alternative approach - plot in batches:

# Plot in batches of 3
batch_size = 3
for i in range(0, len(numerical_columns), batch_size):
    batch = numerical_columns[i:i+batch_size]
    
    plt.figure(figsize=(15, batch_size * 3))
    for j, col in enumerate(batch):
        plt.subplot(batch_size, 2, 2*j+1)
        sns.histplot(df[col], kde=True)
        plt.title(f'Distribution of {col}')
        
        plt.subplot(batch_size, 2, 2*j+2)
        sns.boxplot(x=df[col])
        plt.title(f'Boxplot of {col}')
    
    plt.tight_layout()
    plt.show()

# Calculate skewness and kurtosis for numerical features
skewness = df[numerical_columns].skew().sort_values(ascending=False)
kurtosis = df[numerical_columns].kurt().sort_values(ascending=False)

print("\nSkewness of numerical features:")
display(pd.DataFrame({'Skewness': skewness}))

print("\nKurtosis of numerical features:")
display(pd.DataFrame({'Kurtosis': kurtosis}))

# Create a more detailed box plot visualization using Plotly
fig = make_subplots(rows=len(numerical_columns), cols=1, 
                    subplot_titles=[f'Box Plot of {col}' for col in numerical_columns])

for i, col in enumerate(numerical_columns):
    fig.add_trace(go.Box(x=df[col], name=col), row=i+1, col=1)

fig.update_layout(height=300 * len(numerical_columns), width=900, 
                  title_text="Box Plots of Numerical Features",
                  showlegend=False)
fig.show()

## 5. Categorical Features Analysis

Now let's explore categorical features, their unique values, frequencies, and visualize their distributions.

In [ ]:
# Identify categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical columns: {categorical_columns}")

# Analyze each categorical column
for col in categorical_columns:
    print(f"\nAnalysis of {col}:")
    
    # Count of unique values
    unique_count = df[col].nunique()
    print(f"Number of unique values: {unique_count}")
    
    # Value counts and percentage
    value_counts = df[col].value_counts()
    value_percentage = df[col].value_counts(normalize=True) * 100
    
    # Create a dataframe to display value counts and percentages
    value_stats = pd.DataFrame({
        'Count': value_counts,
        'Percentage (%)': value_percentage.round(2)
    })
    
    # Display the top categories (limit to 10 for readability)
    if len(value_stats) > 10:
        print("Top 10 categories:")
        display(value_stats.head(10))
        print(f"... and {len(value_stats) - 10} more categories")
    else:
        display(value_stats)
    
    # Visualize the distribution if there aren't too many categories
    if unique_count <= 15:  # Only plot if there's a reasonable number of categories
        plt.figure(figsize=(12, 6))
        sns.countplot(y=col, data=df, order=df[col].value_counts().index[:15])
        plt.title(f'Distribution of {col}')
        plt.tight_layout()
        plt.show()
    elif unique_count <= 50:  # For more categories, show horizontal plot with top values
        plt.figure(figsize=(12, 8))
        sns.countplot(y=col, data=df, order=df[col].value_counts().index[:15])
        plt.title(f'Top 15 values of {col}')
        plt.tight_layout()
        plt.show()
    else:
        print(f"Too many unique values ({unique_count}) to visualize {col}.")

## 6. Target Variable Analysis

Let's identify and analyze the target variable (likely salary) distribution and statistics.

In [ ]:
# Assuming the target variable is related to salary and is likely to be a numerical column
# Check for columns that might contain salary information
salary_columns = [col for col in df.columns if any(keyword in col.lower() for keyword in ['salary', 'compensation', 'pay', 'wage', 'income'])]

if salary_columns:
    target_variable = salary_columns[0]  # Use the first salary-related column as the target
    print(f"Identified target variable: {target_variable}")
else:
    # If no salary column is found, assume the last numerical column is the target
    target_variable = numerical_columns[-1]
    print(f"No obvious salary column found. Using {target_variable} as the target variable.")

# Analyze the target variable
print(f"\nAnalysis of target variable: {target_variable}")
print("\nDescriptive Statistics:")
display(df[target_variable].describe())

# Visualize the distribution
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(df[target_variable], kde=True)
plt.title(f'Distribution of {target_variable}')

plt.subplot(1, 2, 2)
sns.boxplot(x=df[target_variable])
plt.title(f'Boxplot of {target_variable}')
plt.tight_layout()
plt.show()
# Make sure your target variable is numeric before calculating quartiles
print(f"Current data type of {target_variable}: {df[target_variable].dtype}")

# If the target variable isn't numeric, try to convert it
if df[target_variable].dtype == 'object':
    print(f"Converting {target_variable} to numeric...")
    # Check a sample of values first
    print("Sample values:")
    print(df[target_variable].head())
    
    # Try to convert to numeric, errors='coerce' will turn non-convertible values into NaN
    df[target_variable] = pd.to_numeric(df[target_variable], errors='coerce')
    print(f"New data type: {df[target_variable].dtype}")
    
    # Check for NaN values after conversion
    nan_count = df[target_variable].isna().sum()
    print(f"NaN values after conversion: {nan_count} ({nan_count/len(df)*100:.2f}%)")

# Now try the quartile calculations again
Q1 = df[target_variable].quantile(0.25)
Q3 = df[target_variable].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df[target_variable] < lower_bound) | (df[target_variable] > upper_bound)]
print(f"\nNumber of outliers in {target_variable}: {len(outliers)}")
print(f"Percentage of outliers: {(len(outliers) / len(df) * 100):.2f}%")

# Interactive histogram using plotly
fig = px.histogram(df, x=target_variable, marginal="box", 
                   title=f"Distribution of {target_variable}",
                   labels={target_variable: target_variable},
                   opacity=0.7)
fig.update_layout(bargap=0.1)
fig.show()

In [ ]:
# Before calculating correlations, check if target_variable exists and is numeric
if target_variable in df.columns:
    print(f"Target variable {target_variable} found in DataFrame")
    if df[target_variable].dtype in ['int64', 'float64']:
        print(f"Target variable is numeric with type {df[target_variable].dtype}")
    else:
        print(f"Target variable is not numeric. Type: {df[target_variable].dtype}")
        print("Converting to numeric...")
        df[target_variable] = pd.to_numeric(df[target_variable], errors='coerce')
else:
    print(f"Target variable {target_variable} not found in DataFrame")
    print(f"Available columns: {df.columns.tolist()}")

# Calculate correlation matrix for numerical features
numerical_df = df.select_dtypes(include=['int64', 'float64'])
corr_matrix = numerical_df.corr()

# Check if target variable is in correlation matrix
if target_variable in corr_matrix.columns:
    # Find features most correlated with the target
    target_correlations = corr_matrix[target_variable].sort_values(ascending=False)
    print("Features most correlated with the target variable:")
    display(target_correlations)
else:
    print(f"Target variable {target_variable} not found in correlation matrix")
    print(f"Correlation matrix columns: {corr_matrix.columns.tolist()}")

# Scatter plots for top correlated features with target variable
top_correlated = [col for col in target_correlations.index if col != target_variable][:5]
if len(top_correlated) > 0:
    plt.figure(figsize=(15, len(top_correlated) * 5))
    for i, feature in enumerate(top_correlated):
        plt.subplot(len(top_correlated), 1, i+1)
        sns.scatterplot(x=df[feature], y=df[target_variable], alpha=0.6)
        plt.title(f'{feature} vs {target_variable}')
        plt.tight_layout()
    plt.show()

    # Create pair plots for highly correlated variables
    top_features = [target_variable] + top_correlated[:3]  # Target plus top 3 correlated features
    print(f"\nCreating pair plot for target and top {len(top_features)-1} correlated features:")
    sns.pairplot(df[top_features], height=2.5)
    plt.suptitle('Pair Plot of Target and Top Correlated Features', y=1.02)
    plt.show()
else:
    print("No correlated features found to create scatter plots.")

# Analyze relationships between categorical features and target
if categorical_columns and target_variable in numerical_columns:
    for cat_col in categorical_columns[:3]:  # Limit to first 3 categorical columns to avoid too many plots
        if df[cat_col].nunique() <= 10:  # Only plot if there aren't too many categories
            plt.figure(figsize=(12, 6))
            sns.boxplot(x=cat_col, y=target_variable, data=df)
            plt.title(f'{cat_col} vs {target_variable}')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()

## 8. Data Visualization

Let's create additional visualizations such as heatmaps, bar charts, and other plots to gain deeper insights into the data.

In [ ]:
# Create a heatmap of feature correlations
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='viridis', fmt='.2f', 
            linewidths=0.5, cbar_kws={"shrink": .8})
plt.title('Feature Correlation Heatmap (Upper Triangle)')
plt.tight_layout()
plt.show()

# Density plots for numerical features
plt.figure(figsize=(15, 10))
for i, col in enumerate(numerical_columns[:6]):  # Plot first 6 numerical columns
    plt.subplot(2, 3, i+1)
    sns.kdeplot(df[col], fill=True)
    plt.title(f'Density Plot of {col}')
plt.tight_layout()
plt.show()

# Create a violin plot comparing distributions
if len(numerical_columns) >= 4:
    plt.figure(figsize=(14, 7))
    sns.violinplot(data=df[numerical_columns[:4]])  # Select first 4 numerical columns
    plt.title('Violin Plot of Numerical Features')
    plt.tight_layout()
    plt.show()

# If we have geographic data, create a scatter plot
geo_columns = [col for col in df.columns if any(keyword in col.lower() for keyword in ['lat', 'lon', 'latitude', 'longitude'])]
if len(geo_columns) >= 2:
    lat_col = [col for col in geo_columns if 'lat' in col.lower()][0]
    lon_col = [col for col in geo_columns if 'lon' in col.lower()][0]
    
    fig = px.scatter_mapbox(df, lat=lat_col, lon=lon_col, color=target_variable,
                           hover_name=df.index, zoom=3)
    fig.update_layout(mapbox_style="open-street-map")
    fig.show()

# Create a 3D scatter plot of the top 3 numeric features
if len(numerical_columns) >= 3:
    fig = px.scatter_3d(df, x=numerical_columns[0], y=numerical_columns[1], z=numerical_columns[2],
                       color=target_variable, opacity=0.7, title="3D Scatter Plot")
    fig.show()

# Generate a parallel coordinates plot for numerical features
if len(numerical_columns) > 1:
    fig = px.parallel_coordinates(df, color=target_variable, 
                                  dimensions=numerical_columns[:7],  # Limit to first 7 columns
                                  color_continuous_scale=px.colors.sequential.Viridis,
                                  title="Parallel Coordinates Plot")
    fig.show()

# Create a radar chart for a sample of the data
if len(numerical_columns) >= 3:
    # Normalize the data for the radar chart
    from sklearn.preprocessing import MinMaxScaler
    
    # Select a sample of 5 records
    sample_df = df.sample(5)
    
    # Normalize the numerical features
    scaler = MinMaxScaler()
    sample_scaled = pd.DataFrame(scaler.fit_transform(sample_df[numerical_columns[:6]]),  # Use first 6 numerical columns
                                columns=numerical_columns[:6])
    
    # Create the radar chart
    categories = numerical_columns[:6]
    fig = go.Figure()
    
    for i in range(len(sample_scaled)):
        fig.add_trace(go.Scatterpolar(
            r=sample_scaled.iloc[i].values,
            theta=categories,
            fill='toself',
            name=f'Record {i+1}'
        ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1]
            )),
        showlegend=True,
        title="Radar Chart of Sample Records"
    )
    fig.show()

## 9. Summary of Findings

Based on our exploratory data analysis, we can summarize the key insights:

1. **Dataset Overview**: 
   - [Note: Complete with actual findings about dataset size, structure, etc.]

2. **Missing Values**: 
   - [Note: Complete with actual findings about missing values]

3. **Numerical Features**: 
   - [Note: Complete with actual findings about numerical distributions]

4. **Categorical Features**: 
   - [Note: Complete with actual findings about categorical variables]

5. **Target Variable**: 
   - [Note: Complete with actual findings about the salary distribution]

6. **Feature Relationships**: 
   - [Note: Complete with actual findings about correlations and relationships]

7. **Key Insights**:
   - [Note: Complete with actual insights based on the analysis]

## 10. Next Steps

Based on our EDA, here are potential next steps:

1. **Feature Engineering**:
   - Create new features based on the insights gained
   - Transform skewed numerical features
   - Encode categorical variables appropriately

2. **Data Preprocessing**:
   - Handle missing values
   - Remove or treat outliers
   - Normalize/standardize features

3. **Model Development**:
   - Select appropriate algorithms for salary prediction
   - Implement cross-validation
   - Evaluate model performance

4. **Further Analysis**:
   - Investigate domain-specific factors affecting engineering salaries
   - Perform more advanced statistical tests
   - Conduct feature importance analysis

## 7. Feature Relationships

Let's examine relationships between features and the target variable using correlation analysis, scatter plots, and pair plots.